<a href="https://colab.research.google.com/github/dgg32/ml_pycaret/blob/main/pycaret_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 11.9MB/s 
     |████████████████████████████████| 1.8MB 18.3MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 276kB 46.8MB/s 
     |████████████████████████████████| 6.8MB 49.4MB/s 
     |████████████████████████████████| 266kB 36.9MB/s 
     |████████████████████████████████| 174kB 53.8MB/s 
     |████████████████████████████████| 1.6MB 39.6MB/s 
     |████████████████████████████████| 148.9MB 83kB/s 
     |████████████████████████████████| 66.3MB 60kB/s 
     |████████████████████████████████| 13.9MB 49.9MB/s 
     |████████████████████████████████| 2.1MB 44.3MB/s 
     |████████████████████████████████| 604kB 43.6MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 3.1MB 36.3MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 


In [2]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


# What is Anomaly Detection?

Anomaly Detection is the task of identifying the rare items, events or observations which raise suspicions by differing significantly from the majority of the data. Typically the anomalous items will translate to some kind of problem such as bank fraud, a structural defect, medical problems or errors in a text. There are three broad categories of anomaly detection techniques exist:

    Unsupervised anomaly detection: Unsupervised anomaly detection techniques detect anomalies in an unlabeled test data set under the assumption that the majority of the instances in the dataset are normal by looking for instances that seem to fit least to the remainder of the data set.

    Supervised anomaly detection: This technique requires a dataset that has been labeled as "normal" and "abnormal" and involves training a classifier.

    Semi-supervised anomaly detection: This technique constructs a model representing normal behavior from a given normal training dataset, and then tests the likelihood of a test instance to be generated by the learnt model.

pycaret.anomaly module supports the unsupervised and supervised anomaly detection technique. In this tutorial we will only cover unsupervised anomaly detection technique.

# Overview of Anomaly Detection Module in PyCaret

PyCaret's anomaly detection module (pycaret.anomaly) is a an unsupervised machine learning module which performs the task of identifying rare items, events or observations which raise suspicions by differing significantly from the majority of the data.

PyCaret anomaly detection module provides several pre-processing features that can be configured when initializing the setup through setup() function. It has over 12 algorithms and few plots to analyze the results of anomaly detection. PyCaret's anomaly detection module also implements a unique function tune_model() that allows you to tune the hyperparameters of anomaly detection model to optimize the supervised learning objective such as AUC for classification or R2 for regression.

# Dataset for the Tutorial

For this tutorial we will use a dataset from UCI called Mice Protein Expression. The dataset consists of the expression levels of 77 proteins/protein modifications that produced detectable signals in the nuclear fraction of cortex. The dataset contains a total of 1080 measurements per protein. Each measurement can be considered as an independent sample/mouse. Click Here to read more about the dataset.

In [3]:
from pycaret.datasets import get_data
dataset = get_data("mice")

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,...,TIAM1_N,pP70S6_N,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,0.687906,0.306382,0.402698,0.296927,1.022060,0.605673,1.877684,2.308745,0.441599,0.859366,0.416289,0.369608,0.178944,1.866358,3.685247,1.537227,0.264526,0.319677,0.813866,0.165846,0.453910,3.037621,0.369510,0.458539,0.335336,0.825192,0.576916,0.448099,0.586271,...,0.482864,0.294170,0.182150,0.842725,0.192608,1.443091,0.294700,0.354605,1.339070,0.170119,0.159102,0.188852,0.106305,0.144989,0.176668,0.125190,0.115291,0.228043,0.142756,0.430957,0.247538,1.603310,2.014875,0.108234,1.044979,0.831557,0.188852,0.122652,NaN,0.106305,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
1,309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,0.695006,0.299051,0.385987,0.281319,0.956676,0.587559,1.725774,2.043037,0.445222,0.834659,0.400364,0.356178,0.173680,1.761047,3.485287,1.509249,0.255727,0.304419,0.780504,0.157194,0.430940,2.921882,0.342279,0.423560,0.324835,0.761718,0.545097,0.420876,0.545097,...,0.454519,0.276431,0.182086,0.847615,0.194815,1.439460,0.294060,0.354548,1.306323,0.171427,0.158129,0.184570,0.106592,0.150471,0.178309,0.134275,0.118235,0.238073,0.142037,0.457156,0.257632,1.671738,2.004605,0.109749,1.009883,0.849270,0.200404,0.116682,NaN,0.106592,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
2,309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,0.677348,0.291276,0.381002,0.281710,1.003635,0.602449,1.731873,2.017984,0.467668,0.814329,0.399847,0.368089,0.173905,1.765544,3.571456,1.501244,0.259614,0.311747,0.785154,0.160895,0.423187,2.944136,0.343696,0.425005,0.324852,0.757031,0.543620,0.404630,0.552994,...,0.447197,0.256648,0.184388,0.856166,0.200737,1.524364,0.301881,0.386087,1.279600,0.185456,0.148696,0.190532,0.108303,0.145330,0.176213,0.132560,0.117760,0.244817,0.142445,0.510472,0.255343,1.663550,2.016831,0.108196,0.996848,0.846709,0.193685,0.118508,NaN,0.108303,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
3,309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,0.583277,0.296729,0.377087,0.313832,0.875390,0.520293,1.566852,2.132754,0.477671,0.727705,0.385639,0.362970,0.179449,1.286277,2.970137,1.419710,0.259536,0.279218,0.734492,0.162210,0.410615,2.500204,0.344509,0.429211,0.330121,0.746980,0.546763,0.386860,0.547849,...,0.442650,0.398534,0.161768,0.760234,0.184169,1.612382,0.296382,0.290680,1.198765,0.159799,0.166112,0.185323,0.103184,0.140656,0.163804,0.123210,0.117439,0.234947,0.145068,0.430996,0.251103,1.484624,1.957233,0.119883,0.990225,0.833277,0.192112,0.132781,NaN,0.103184,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
4,309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,0.550960,0.286961,0.363502,0.277964,0.864912,0.507990,1.480059,2.013697,0.483416,0.687794,0.367531,0.355311,0.174836,1.324695,2.896334,1.359876,0.250705,0.273667,0.702699,0.154827,0.398550,2.456560,0.329126,0.408755,0.313415,0.691956,0.536860,0.360816,0.512824,...,0.419095,0.393447,0.160200,0.768113,0.185718,1.645807,0.296829,0.309345,1.206995,0.164650,0.160687,0.188221,0.104784,0.141983,0.167710,0.136838,0.116048,0.255528,0.140871,0.48

In [4]:
dataset.shape

(1080, 82)

In order to demonstrate the predict_model() function on unseen data, a sample of 5% (54 samples) are taken out from original dataset to be used for predictions at the end of experiment. This should not be confused with train/test split. This particular split is performed to simulate real life scenario. Another way to think about this is that these 54 samples are not available at the time when this experiment was performed.

In [5]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print (f"Data for modeling {data.shape}")
print (f"Unseen Data for predictions: {data_unseen.shape}")

Data for modeling (1026, 82)
Unseen Data for predictions: (54, 82)


# Setting up Environment in PyCaret

setup() function initializes the environment in PyCaret and creates the transformation pipeline to prepare the data for modeling and deployment. setup() must be called before executing any other function in PyCaret. It takes only one mandatory parameter: pandas dataframe. All other parameters are optional and are used to customize pre-processing pipeline (we will see them in later tutorials).

When setup() is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties. Although, most of the times the data type is inferred correctly but it's not always the case. Therefore, after setup() is executed, PyCaret displays a table containing features and their inferred data types. At which stage, you can inspect and press enter to continue if all data types are correctly inferred or type quit to end the experiment. Identifying data types correctly is of fundamental importance in PyCaret as it automatically performs few pre-processing tasks which are imperative to perform any machine learning experiment. These pre-processing tasks are performed differently for each data type. As such, it is very important that data types are correctly configured.

In later tutorials we will learn how to overwrite PyCaret's inferred data types using numeric_features and categorical_features parameter in setup()

In [6]:
from pycaret.anomaly import *

exp_ano101 = setup(data, normalize=True, ignore_features=["MouseID"], session_id=123)

,Description,Value
0,session_id,123
1,Original Data,"(1026, 82)"
2,Missing Values,True
3,Numeric Features,77
4,Categorical Features,4
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1026, 91)"
9,CPU Jobs,-1




Once the setup is succesfully executed it prints the information grid that contains few important information. Much of the information is related to pre-processing pipeline which is constructed when setup() is executed. Much of these features are out of scope for the purpose of this tutorial. However, few important things to note at this stage are:

    session_id : A pseudo-random number distributed as a seed in all functions for later reproducibility. If no session_id is passed, a random number is automatically generated that is distributed to all functions. In this experiment session_id is set as 123 for later reproducibility.

    Missing Values : When there are missing values in original data it will show as True. Notice that Missing Values in the information grid above is True as the data contains missing values which are automatically imputed using mean for numeric features and constant for categorical features. The method of imputation can be changed using numeric_imputation and categorical_imputation parameter in setup().

    Original Data : Displays the original shape of dataset. In this experiment (1026, 82) means 1026 samples and 82 features.

    Transformed Data : Displays the shape of transformed dataset. Notice that the shape of original dataset (1026, 82) is transformed into (1026, 91). The number of features has increased due to encoding of categorical features in the dataset.

    Numeric Features : Number of features inferred as numeric. In this dataset, 77 out of 82 features are inferred as numeric.

    Categorical Features : Number of features inferred as categorical. In this dataset, 5 out of 82 features are inferred as categorical. Also notice, we have ignored one categorical feature i.e. MouseID using ignore_feature parameter.

Notice that how few tasks such as missing value imputation and categorical encoding that are imperative to perform modeling are automatically handled. Most of the other parameters in setup() are optional and used for customizing pre-processing pipeline. These parameters are out of scope for this tutorial but as you progress to intermediate and expert level, we will cover them in much detail.


# Create a Model

Creating an anomaly detection model in PyCaret is simple and similar to how you would have created a model in supervised modules of PyCaret. The anomaly detection model is created using create_model() function which takes one mandatory parameter i.e. name of the model as a string. This function returns a trained model object. See the example below:

In [7]:
iforest = create_model("iforest")

In [8]:
print (iforest)

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)


We have created Isolation Forest model using create_model(). Notice the contamination parameter is set 0.05 which is the default value when you do not pass fraction parameter in create_model(). fraction parameter determines the proportion of outliers in the dataset. In below example, we will create One Class Support Vector Machine model with 0.025 fraction.

In [9]:
svm = create_model("svm", fraction=0.025)

In [10]:
print (svm)

OCSVM(cache_size=200, coef0=0.0, contamination=0.025, degree=3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)




Just by replacing iforest with svm inside create_model() we have now created OCSVM anomaly detection model. There are 12 models available ready-to-use in pycaret.anomaly module. To see the complete list, please see docstring or use models function.


In [11]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


# Assign a Model

Now that we have created a model, we would like to assign the anomaly labels to our dataset (1080 samples) to analyze the results. We will achieve this by using assign_model() function. See an example below:

In [12]:
iforest_results = assign_model(iforest)
iforest_results.head()

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,...,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,Anomaly,Anomaly_Score
0,3501_12,0.344930,0.626194,0.383583,2.534561,4.097317,0.303547,0.222829,4.592769,0.239427,1.360164,0.339472,0.360164,0.333788,0.335380,0.926330,0.827876,1.839700,1.256708,0.459982,0.871305,0.298317,0.467030,0.219873,1.283765,2.598454,1.177126,0.287176,0.331060,0.793543,0.195771,0.512051,2.261255,0.815598,0.483856,0.503638,0.893815,0.732378,0.438154,0.720327,...,0.188809,0.880156,0.160182,1.521666,0.300325,0.642811,1.414053,0.339493,0.352115,0.201952,0.140664,0.187638,0.223943,0.327651,0.124528,0.182824,0.112687,0.676122,0.331685,1.510345,2.378920,0.192453,1.897202,0.962785,0.283409,0.150683,0.163175,0.140664,0.151464,0.455172,0.252700,0.218868,0.249187,1.139493,Ts65Dn,Memantine,S/C,t-SC-m,0,-0.014462
1,3520_5,0.630001,0.839187,0.357777,2.651229,4.261675,0.253184,0.185257,3.816673,0.204940,1.716583,0.825807,0.338737,0.403705,0.305030,0.881256,0.697285,1.691368,2.214460,0.561688,0.680690,0.598482,0.358163,0.182040,1.273125,2.754664,1.506497,0.258459,0.288820,0.837643,0.167246,0.416699,2.310305,0.324585,0.451692,0.408079,0.773061,0.510485,0.373086,0.628586,...,0.209459,1.134340,0.169316,2.388911,0.310546,0.683029,1.580882,0.274112,0.152557,0.171966,0.135334,0.166534,0.228140,0.263447,0.125729,0.235493,0.124735,0.569356,0.327636,1.964494,2.422032,0.151100,1.286831,1.022523,0.228736,0.151166,0.107909,0.135334,0.109897,0.496423,0.155008,0.153219,NaN,1.642886,Control,Memantine,C/S,c-CS-m,0,-0.070193
2,3414_13,0.555122,0.726229,0.278319,2.097249,2.897553,0.222222,0.174356,1.867880,0.203379,1.610136,0.686593,0.259692,0.306043,0.239116,0.680745,0.385532,1.213775,1.433398,0.454191,0.499025,0.617934,0.315789,0.187568,0.800520,1.991553,1.323587,0.248213,0.246047,0.736409,0.172623,0.412606,1.916613,0.316223,0.365389,0.285034,0.602339,0.176522,0.289149,0.268789,...,0.144503,0.800527,0.164417,1.922153,0.247202,0.319454,0.748190,0.143680,0.144997,0.156353,0.104839,0.144338,0.158492,0.184661,0.104016,0.195523,0.101876,0.435813,0.272383,1.195523,2.536537,0.140882,1.004773,0.936307,0.200790,0.152732,NaN,0.104839,NaN,0.344964,0.136109,0.155530,0.185484,1.657670,Ts65Dn,Memantine,C/S,t-CS-m,0,-0.070143
3,3488_8,0.275849,0.430764,0.285166,2.265254,3.250091,0.189258,0.157837,2.917611,0.202594,1.734746,0.321703,0.262148,0.262696,0.229083,0.763062,0.744428,1.607782,1.037998,0.378882,0.767994,0.245342,0.365181,0.155645,1.199854,2.045305,0.900804,0.214468,0.228718,0.580197,0.153453,0.350018,2.036536,1.012788,0.477347,0.513336,0.723054,0.540921,0.362258,0.630252,...,0.154218,0.813469,0.133884,0.986310,0.225388,0.268673,0.917556,0.108516,0.140125,0.151299,0.123415,0.148581,0.165895,0.163479,0.111033,0.236964,0.110630,0.526676,0.259211,1.226696,2.146165,0.164586,1.065331,0.704349,0.247332,0.142138,0.126636,0.123415,0.147373,0.390880,0.127944,0.207671,0.175357,0.893598,Control,Saline,S/C,c-SC-s,0,-0.080521
4,3501_7,0.304788,0.617299,0.335164,2.638236,4.876609,0.280590,0.199417,4.835421,0.236314,1.226532,0.309765,0.365711,0.336537,0.318689,0.902351,0.805389,1.845203,1.380470,0.438476,0.929810,0.270808,0.433499,0.190836,1.308907,2.981122,1.155140,0.265145,0.294148,0.801785,0.181397,0.466793,2.484984,0.765402,0.465076,0.458212,0.904925,0.689720,0.392826,0.672044,...,0.189124,1.029642,0.154055,1.403507,0.281599,0.565285,1.266361,0.366559,0.151759,0.182966,0.149567,0.181923,0.217096,0.301743,0.112619,0.205511,0.112097,0.624883,0.308736,1.577393,2.47729

Notice that two columns Label and Score are added towards the end. 0 stands for inliers and 1 for outliers/anomalies. Score is the values computed by the algorithm. Outliers are assigned with larger anomaly scores. Notice that iforest_results also includes MouseID feature that we have dropped during setup(). It wasn't used for the model and is only appended to the dataset when you use assign_model(). In the next section we will see how to analyze the results of anomaly detection using plot_model().

# plot a model

plot_model() function can be used to analyze the anomaly detection model over different aspects. This function takes a trained model object and returns a plot. See the examples below:

In [15]:
plot_model(iforest)

In [14]:
plot_model(iforest, plot="umap")

# Predict on Unseen Data

predict_model() function is used to assign anomaly labels on the new unseen dataset. We will now use our iforest model to predict the data stored in data_unseen. This was created in the beginning of the experiment and it contains 54 new samples that were not exposed to PyCaret before.

In [17]:
unseen_predictions = predict_model(iforest, data=data_unseen)
unseen_predictions.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,pNUMB_N,...,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,Anomaly,Anomaly_Score
0,0.447506,0.628176,0.367388,2.385939,4.807635,0.218578,0.176233,2.141282,0.195188,1.442398,0.566340,0.289824,0.363893,0.266837,0.859121,0.521307,1.538244,1.968275,0.495900,0.672402,0.369404,0.357172,0.179728,1.227450,2.956983,1.447910,0.250840,0.284044,0.704396,0.156876,0.391047,2.467133,0.327598,0.404490,0.296276,0.674419,0.539723,0.354214,0.514316,0.347224,...,0.162330,0.779695,0.186793,1.634615,0.288037,0.332367,1.123445,0.175693,0.150594,0.183824,0.106476,0.139564,0.174844,0.130515,0.115243,0.236850,0.136454,0.478577,0.244485,1.507777,2.003535,0.120687,0.920178,0.843679,0.190469,0.131575,NaN,0.106476,0.109446,0.439833,0.116657,0.140766,0.142180,1.816389,Control,Memantine,C/S,c-CS-m,0,-0.077131
1,0.704633,0.802537,0.350110,2.467733,5.548400,0.205323,0.165058,2.107281,0.171401,1.938913,1.065637,0.326669,0.342113,0.243243,0.894374,0.742554,1.757170,1.521098,0.388307,0.812328,0.528130,0.362521,0.165609,1.345422,3.684915,1.378240,0.243105,0.275097,0.739520,0.154716,0.393547,2.415472,0.353144,0.491175,0.348731,0.831081,0.611142,0.457391,0.582322,0.457253,...,0.183455,0.885730,0.196724,0.975494,0.281005,0.400961,1.176618,0.098362,0.183861,0.186705,0.112036,0.139859,0.152383,0.185825,0.133563,0.230233,0.129637,0.388302,0.237544,1.147983,1.791836,0.136339,1.044476,0.713851,0.174249,0.131126,0.132887,0.112036,NaN,0.454509,0.111089,0.157731,0.158543,1.404481,Control,Memantine,C/S,c-CS-m,0,-0.060165
2,0.505093,0.695549,0.376029,2.915585,5.917957,0.226734,0.174271,2.663039,0.190038,1.535091,0.719967,0.327055,0.329287,0.255616,1.015767,0.796847,2.011302,1.601646,0.398214,0.858100,0.374494,0.402121,0.164225,1.682712,3.815404,1.419143,0.266499,0.282545,0.829496,0.163667,0.408121,2.538161,0.329427,0.520022,0.410353,0.798102,0.649644,0.458769,0.697502,0.471885,...,0.236385,1.193611,0.171562,0.925127,0.337359,0.503779,1.813167,0.105259,0.163537,0.199533,0.151928,0.189871,0.212855,0.272536,0.132606,0.240514,0.135411,0.598987,0.309155,1.745695,2.579899,0.167199,1.322400,0.916245,0.216362,NaN,0.126451,0.151928,NaN,0.591586,0.131515,0.188391,NaN,1.699260,Control,Memantine,C/S,c-CS-m,0,-0.052132
3,0.429133,0.563175,0.258429,2.028151,3.542553,0.214075,0.176759,3.165139,0.167430,1.217676,0.579214,0.300491,0.315548,0.286579,0.753028,0.659574,1.331588,1.717676,0.476759,0.546645,0.361538,0.304092,0.163011,0.897872,1.912602,1.125205,0.231915,0.241571,0.602946,0.132733,0.334043,1.714239,0.271522,0.445990,0.425041,0.639444,0.505892,0.356301,0.528642,0.343044,...,0.170544,0.777029,0.153490,2.039267,0.264449,0.365617,1.315717,0.104643,0.144752,0.162438,0.131593,0.167386,0.184862,0.174334,0.128435,0.245605,0.105695,0.540373,0.328245,1.444468,2.247816,0.172229,1.133804,0.937467,0.243078,0.167912,0.120960,0.131593,0.143068,0.371618,0.118223,0.171071,0.173702,1.405727,Control,Memantine,C/S,c-CS-m,0,-0.091610
4,0.373648,0.471165,0.257909,1.860032,2.938526,0.218262,0.150380,2.610132,0.142571,1.020024,0.492591,0.276131,0.298158,0.250300,0.718062,0.596916,1.213056,1.727673,0.476572,0.540449,0.317181,0.255106,0.166199,0.761914,1.571886,1.012615,0.210052,0.187825,0.546856,0.130557,0.352823,1.494393,0.260913,0.482980,0.475771,0.659992,0.504405,0.357829,0.573889,0.334602,...,0.149576,0.471171,0.125463,1.843739,0.237794,0.264653,0.840754,0.075564,0.159842,0.102304,0.084398,0.117226,0.119494,0.100155,0.118181,0.128208,0.072580,0.374955,0.224663,0.971589,1.32

Label column indicates the outlier (1 = outlier, 0 = inlier). Score is the values computed by the algorithm. Outliers are assigned with larger anomaly scores. You can also use predict_model() function to label the training data. See example below:

In [18]:
data_predictions = predict_model(iforest, data=data)
data_predictions.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,pNUMB_N,...,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,Anomaly,Anomaly_Score
0,0.344930,0.626194,0.383583,2.534561,4.097317,0.303547,0.222829,4.592769,0.239427,1.360164,0.339472,0.360164,0.333788,0.335380,0.926330,0.827876,1.839700,1.256708,0.459982,0.871305,0.298317,0.467030,0.219873,1.283765,2.598454,1.177126,0.287176,0.331060,0.793543,0.195771,0.512051,2.261255,0.815598,0.483856,0.503638,0.893815,0.732378,0.438154,0.720327,0.407003,...,0.188809,0.880156,0.160182,1.521666,0.300325,0.642811,1.414053,0.339493,0.352115,0.201952,0.140664,0.187638,0.223943,0.327651,0.124528,0.182824,0.112687,0.676122,0.331685,1.510345,2.378920,0.192453,1.897202,0.962785,0.283409,0.150683,0.163175,0.140664,0.151464,0.455172,0.252700,0.218868,0.249187,1.139493,Ts65Dn,Memantine,S/C,t-SC-m,0,-0.014462
1,0.630001,0.839187,0.357777,2.651229,4.261675,0.253184,0.185257,3.816673,0.204940,1.716583,0.825807,0.338737,0.403705,0.305030,0.881256,0.697285,1.691368,2.214460,0.561688,0.680690,0.598482,0.358163,0.182040,1.273125,2.754664,1.506497,0.258459,0.288820,0.837643,0.167246,0.416699,2.310305,0.324585,0.451692,0.408079,0.773061,0.510485,0.373086,0.628586,0.394700,...,0.209459,1.134340,0.169316,2.388911,0.310546,0.683029,1.580882,0.274112,0.152557,0.171966,0.135334,0.166534,0.228140,0.263447,0.125729,0.235493,0.124735,0.569356,0.327636,1.964494,2.422032,0.151100,1.286831,1.022523,0.228736,0.151166,0.107909,0.135334,0.109897,0.496423,0.155008,0.153219,NaN,1.642886,Control,Memantine,C/S,c-CS-m,0,-0.070193
2,0.555122,0.726229,0.278319,2.097249,2.897553,0.222222,0.174356,1.867880,0.203379,1.610136,0.686593,0.259692,0.306043,0.239116,0.680745,0.385532,1.213775,1.433398,0.454191,0.499025,0.617934,0.315789,0.187568,0.800520,1.991553,1.323587,0.248213,0.246047,0.736409,0.172623,0.412606,1.916613,0.316223,0.365389,0.285034,0.602339,0.176522,0.289149,0.268789,0.273338,...,0.144503,0.800527,0.164417,1.922153,0.247202,0.319454,0.748190,0.143680,0.144997,0.156353,0.104839,0.144338,0.158492,0.184661,0.104016,0.195523,0.101876,0.435813,0.272383,1.195523,2.536537,0.140882,1.004773,0.936307,0.200790,0.152732,NaN,0.104839,NaN,0.344964,0.136109,0.155530,0.185484,1.657670,Ts65Dn,Memantine,C/S,t-CS-m,0,-0.070143
3,0.275849,0.430764,0.285166,2.265254,3.250091,0.189258,0.157837,2.917611,0.202594,1.734746,0.321703,0.262148,0.262696,0.229083,0.763062,0.744428,1.607782,1.037998,0.378882,0.767994,0.245342,0.365181,0.155645,1.199854,2.045305,0.900804,0.214468,0.228718,0.580197,0.153453,0.350018,2.036536,1.012788,0.477347,0.513336,0.723054,0.540921,0.362258,0.630252,0.279503,...,0.154218,0.813469,0.133884,0.986310,0.225388,0.268673,0.917556,0.108516,0.140125,0.151299,0.123415,0.148581,0.165895,0.163479,0.111033,0.236964,0.110630,0.526676,0.259211,1.226696,2.146165,0.164586,1.065331,0.704349,0.247332,0.142138,0.126636,0.123415,0.147373,0.390880,0.127944,0.207671,0.175357,0.893598,Control,Saline,S/C,c-SC-s,0,-0.080521
4,0.304788,0.617299,0.335164,2.638236,4.876609,0.280590,0.199417,4.835421,0.236314,1.226532,0.309765,0.365711,0.336537,0.318689,0.902351,0.805389,1.845203,1.380470,0.438476,0.929810,0.270808,0.433499,0.190836,1.308907,2.981122,1.155140,0.265145,0.294148,0.801785,0.181397,0.466793,2.484984,0.765402,0.465076,0.458212,0.904925,0.689720,0.392826,0.672044,0.398318,...,0.189124,1.029642,0.154055,1.403507,0.281599,0.565285,1.266361,0.366559,0.151759,0.182966,0.149567,0.181923,0.217096,0.301743,0.112619,0.205511,0.112097,0.624883,0.308736,1.577393

# Saving the Model

We have now finished the experiment by using our iforest model to predict outlier labels on unseen data. This brings us to the end of our experiment but one question is still to be asked. What happens when you have more new data to predict? Do you have to go through the entire experiment again? The answer is No, you don't need to rerun the entire experiment and reconstruct the pipeline to generate predictions on new data. PyCaret's inbuilt function save_model() allows you to save the model along with entire transformation pipeline for later use.

In [19]:
save_model(iforest, "Final IForest Model 18Nov2020")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True,
                                       features_todrop=['MouseID'],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_n...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('

# Loading the Saved Model

To load a saved model on a future date in the same or different environment, we would use the PyCaret's load_model() function and then easily apply the saved model on new unseen data for prediction

In [20]:
saved_iforest = load_model("Final IForest Model 18Nov2020")

Transformation Pipeline and Model Successfully Loaded


Once the model is loaded in the environment, you can simply use it to predict on any new data using the same predict_model() function . Below we have applied the loaded model to predict the same data_unseen that we have used in section 10 above.

In [21]:
new_prediction = predict_model(saved_iforest, data = data_unseen)
new_prediction.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,pNUMB_N,...,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,Anomaly,Anomaly_Score
0,0.447506,0.628176,0.367388,2.385939,4.807635,0.218578,0.176233,2.141282,0.195188,1.442398,0.566340,0.289824,0.363893,0.266837,0.859121,0.521307,1.538244,1.968275,0.495900,0.672402,0.369404,0.357172,0.179728,1.227450,2.956983,1.447910,0.250840,0.284044,0.704396,0.156876,0.391047,2.467133,0.327598,0.404490,0.296276,0.674419,0.539723,0.354214,0.514316,0.347224,...,0.162330,0.779695,0.186793,1.634615,0.288037,0.332367,1.123445,0.175693,0.150594,0.183824,0.106476,0.139564,0.174844,0.130515,0.115243,0.236850,0.136454,0.478577,0.244485,1.507777,2.003535,0.120687,0.920178,0.843679,0.190469,0.131575,NaN,0.106476,0.109446,0.439833,0.116657,0.140766,0.142180,1.816389,Control,Memantine,C/S,c-CS-m,0,-0.077131
1,0.704633,0.802537,0.350110,2.467733,5.548400,0.205323,0.165058,2.107281,0.171401,1.938913,1.065637,0.326669,0.342113,0.243243,0.894374,0.742554,1.757170,1.521098,0.388307,0.812328,0.528130,0.362521,0.165609,1.345422,3.684915,1.378240,0.243105,0.275097,0.739520,0.154716,0.393547,2.415472,0.353144,0.491175,0.348731,0.831081,0.611142,0.457391,0.582322,0.457253,...,0.183455,0.885730,0.196724,0.975494,0.281005,0.400961,1.176618,0.098362,0.183861,0.186705,0.112036,0.139859,0.152383,0.185825,0.133563,0.230233,0.129637,0.388302,0.237544,1.147983,1.791836,0.136339,1.044476,0.713851,0.174249,0.131126,0.132887,0.112036,NaN,0.454509,0.111089,0.157731,0.158543,1.404481,Control,Memantine,C/S,c-CS-m,0,-0.060165
2,0.505093,0.695549,0.376029,2.915585,5.917957,0.226734,0.174271,2.663039,0.190038,1.535091,0.719967,0.327055,0.329287,0.255616,1.015767,0.796847,2.011302,1.601646,0.398214,0.858100,0.374494,0.402121,0.164225,1.682712,3.815404,1.419143,0.266499,0.282545,0.829496,0.163667,0.408121,2.538161,0.329427,0.520022,0.410353,0.798102,0.649644,0.458769,0.697502,0.471885,...,0.236385,1.193611,0.171562,0.925127,0.337359,0.503779,1.813167,0.105259,0.163537,0.199533,0.151928,0.189871,0.212855,0.272536,0.132606,0.240514,0.135411,0.598987,0.309155,1.745695,2.579899,0.167199,1.322400,0.916245,0.216362,NaN,0.126451,0.151928,NaN,0.591586,0.131515,0.188391,NaN,1.699260,Control,Memantine,C/S,c-CS-m,0,-0.052132
3,0.429133,0.563175,0.258429,2.028151,3.542553,0.214075,0.176759,3.165139,0.167430,1.217676,0.579214,0.300491,0.315548,0.286579,0.753028,0.659574,1.331588,1.717676,0.476759,0.546645,0.361538,0.304092,0.163011,0.897872,1.912602,1.125205,0.231915,0.241571,0.602946,0.132733,0.334043,1.714239,0.271522,0.445990,0.425041,0.639444,0.505892,0.356301,0.528642,0.343044,...,0.170544,0.777029,0.153490,2.039267,0.264449,0.365617,1.315717,0.104643,0.144752,0.162438,0.131593,0.167386,0.184862,0.174334,0.128435,0.245605,0.105695,0.540373,0.328245,1.444468,2.247816,0.172229,1.133804,0.937467,0.243078,0.167912,0.120960,0.131593,0.143068,0.371618,0.118223,0.171071,0.173702,1.405727,Control,Memantine,C/S,c-CS-m,0,-0.091610
4,0.373648,0.471165,0.257909,1.860032,2.938526,0.218262,0.150380,2.610132,0.142571,1.020024,0.492591,0.276131,0.298158,0.250300,0.718062,0.596916,1.213056,1.727673,0.476572,0.540449,0.317181,0.255106,0.166199,0.761914,1.571886,1.012615,0.210052,0.187825,0.546856,0.130557,0.352823,1.494393,0.260913,0.482980,0.475771,0.659992,0.504405,0.357829,0.573889,0.334602,...,0.149576,0.471171,0.125463,1.843739,0.237794,0.264653,0.840754,0.075564,0.159842,0.102304,0.084398,0.117226,0.119494,0.100155,0.118181,0.128208,0.072580,0.374955,0.224663,0.971589,1.32

Notice that results of unseen_predictions and new_prediction are identical.